<a href="https://colab.research.google.com/github/ganeshIIT/airbnb/blob/master/airbnb_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
import pyodbc

import helper
import dataloader

In [2]:
apikey = helper.get_apikey()

In [3]:
cstring = helper.get_connstring()
with pyodbc.connect(cstring) as conn:
    query = '''select id, original_language, original_title, overview,
        adult, backdrop_path, budget, homepage, imdb_id, 
       popularity, poster_path, release_date, revenue, runtime,
       status, tagline, title, video,
       vote_average, vote_count, [belongs_to_collection.id]
       from moviesraw'''
    df_movies = pd.read_sql(query, conn)

/var/folders/74/ndv7v28d3078l2cq22t43pkc0000gn/T/ipykernel_13814/169987325.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_movies = pd.read_sql(query, conn)


In [60]:
df_movies= df_movies[['id', 'original_language', 'original_title', 'overview',
          'adult', 'backdrop_path', 'budget', 'homepage', 'imdb_id', 
       'popularity', 'poster_path', 'release_date', 'revenue', 'runtime',
        'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'belongs_to_collection.id']]

In [61]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182643 entries, 0 to 182642
Data columns (total 21 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   id                        182643 non-null  int64         
 1   original_language         182643 non-null  object        
 2   original_title            182643 non-null  object        
 3   overview                  182643 non-null  object        
 4   adult                     182643 non-null  bool          
 5   backdrop_path             80593 non-null   object        
 6   budget                    182643 non-null  int64         
 7   homepage                  182643 non-null  object        
 8   imdb_id                   177268 non-null  object        
 9   popularity                182642 non-null  float64       
 10  poster_path               138576 non-null  object        
 11  release_date              178395 non-null  datetime64[ns]
 12  re

In [62]:
df_movies = (
    df_movies
            # .query('id.str.isnumeric()==True')
            .assign(
                # id = lambda x: x['id'].astype('int'),
                # adult = lambda x: x['adult'].apply(lambda y: 1 if (y=='1' or y=='True') else 0).astype('boolean'),
                release_date = lambda x: pd.to_datetime(x['release_date'].replace({0:np.NaN})).dt.normalize(),
                # budget = lambda x: x['budget'].fillna(0).astype('float').astype('int'),
                # revenue = lambda x: x['revenue'].fillna(0).astype('float').astype('int'),
                # runtime = lambda x: x['runtime'].fillna(0).astype('float').astype('int'),
                # popularity = lambda x: x['popularity'].astype('float'),
                # vote_average = lambda x: x['vote_average'].astype('float'),
                # vote_count = lambda x: x['vote_count'].fillna(0).astype('float').astype('int'),
                    )
)

In [63]:
custom = {"id": "int PRIMARY KEY", "revenue": "bigint", "budget": "bigint", "release_date": "DATE"}
dataloader.full_load_with_index(df=df_movies, tbl="Movies", custom=custom)

DataFrame full loaded to Table: Movies


In [43]:
cstring = helper.get_connstring()
with pyodbc.connect(cstring) as conn:
    query = "select id, genres from moviesraw"
    df_genres = pd.read_sql(query, conn)

/var/folders/74/ndv7v28d3078l2cq22t43pkc0000gn/T/ipykernel_29871/2613549318.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_genres = pd.read_sql(query, conn)


In [44]:
df_genres = (
   df_genres.assign(genres = lambda x: x['genres'].apply(lambda x: literal_eval(x)))
      .explode('genres')
      .dropna()
      .assign(genres = lambda x: x['genres'].apply(lambda y: y['id']))
      .rename(columns={'id':'movieid', 'genres': 'genreid'})
      .drop_duplicates()
)

In [64]:
# custom = {"movieid": "int PRIMARY KEY", "genreid": "int PRIMARY KEY"}
custom = {"movieid": "int not null", "genreid": "int not null"}
dataloader.full_load_with_index(df=df_genres, tbl="MovieGenres", custom=custom)

DataFrame full loaded to Table: MovieGenres


In [46]:
cstring = helper.get_connstring()
with pyodbc.connect(cstring) as conn:
    query = "select id, production_companies from moviesraw"
    df_companies = pd.read_sql(query, conn)

/var/folders/74/ndv7v28d3078l2cq22t43pkc0000gn/T/ipykernel_29871/4003148370.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_companies = pd.read_sql(query, conn)


In [48]:
df_companies = (
   df_companies.assign(production_companies = lambda x: x['production_companies'].apply(lambda x: literal_eval(str(x))))
       .explode('production_companies')
       .dropna()
       .assign(production_companies = lambda x: x['production_companies'].apply(lambda y: y['id']))
       .rename(columns={'id':'movieid', 'production_companies': 'productioncompanyid'})
       .drop_duplicates()
)

In [18]:
df_companies

,movieid,productioncompanyid
0,2,2303
1,3,2303
2,5,14
2,5,59
3,6,182
...,...,...
182638,426928,749
182639,426941,81715
182640,426942,110436
182642,426959,11617


In [49]:
custom = {"movieid": "int not null", "productioncompanyid": "int not null"}
dataloader.full_load_with_index(df=df_companies, tbl="ProductionCompanies", custom=custom)

DataFrame full loaded to Table: ProductionCompanies


In [50]:
cstring = helper.get_connstring()
with pyodbc.connect(cstring) as conn:
    query = "select id, production_countries from moviesraw"
    df_countries = pd.read_sql(query, conn)

/var/folders/74/ndv7v28d3078l2cq22t43pkc0000gn/T/ipykernel_29871/2612712173.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_countries = pd.read_sql(query, conn)


In [52]:
df_countries=(
   df_countries.assign(production_countries = lambda x: x['production_countries'].apply(lambda x: literal_eval(str(x))))
       .explode('production_countries')
       .dropna()
       .assign(production_countries = lambda x: x['production_countries'].apply(lambda y: y['name']))
       .rename(columns={'id':'movieid', 'production_countries':'productioncountry'})
       .drop_duplicates()
)

In [53]:
df_countries

,movieid,productioncountry
0,2,Finland
1,3,Finland
2,5,United States of America
3,6,United States of America
4,8,Austria
...,...,...
182638,426928,France
182639,426941,Spain
182640,426942,Romania
182641,426945,Honduras


In [54]:
custom = {"movieid": "int not null", "productioncountry": "varchar(255) not null"}
dataloader.full_load_with_index(df=df_countries, tbl="ProductionCountries", custom=custom)

DataFrame full loaded to Table: ProductionCountries


In [55]:
cstring = helper.get_connstring()
with pyodbc.connect(cstring) as conn:
    query = "select id, spoken_languages from moviesraw"
    df_spoken = pd.read_sql(query, conn)

/var/folders/74/ndv7v28d3078l2cq22t43pkc0000gn/T/ipykernel_29871/2513746654.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_spoken = pd.read_sql(query, conn)


In [56]:
df_spoken= (
   df_spoken.assign(spoken_languages = lambda x: x['spoken_languages'].apply(lambda x: literal_eval(str(x))))
       .explode('spoken_languages')
       .dropna()
       .assign(spoken_languages = lambda x: x['spoken_languages'].apply(lambda y: y['english_name']))
       .rename(columns={'id':'movieid', 'spoken_languages':'spokenlanguage'})
       .drop_duplicates()
)

In [58]:
custom = {"movieid": "int not null", "spokenlanguage": "varchar(255) not null"}
dataloader.full_load_with_index(df=df_spoken, tbl="SpokenLanguages", custom=custom)

DataFrame full loaded to Table: SpokenLanguages


In [ ]:
# ALTER TABLE Genres
# ADD CONSTRAINT PK_movie_genre PRIMARY KEY (movieid,genreid);

# ALTER TABLE SpokenLanguages
# ADD CONSTRAINT PK_movie_language PRIMARY KEY (movieid,spokenlanguage);

# ALTER TABLE ProductionCompanies
# ADD CONSTRAINT PK_movie_company PRIMARY KEY (movieid,productioncompanyid);

# ALTER TABLE ProductionCountries
# ADD CONSTRAINT PK_movie_country PRIMARY KEY (movieid,productioncountry);

In [90]:
with pyodbc.connect(cstring) as conn:
        query = "select id from movieids"
        dbmovieids = pd.read_sql(query, conn)

/var/folders/74/ndv7v28d3078l2cq22t43pkc0000gn/T/ipykernel_29871/1804200230.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dbmovieids = pd.read_sql(query, conn)


In [91]:
from datetime import datetime, timedelta
fetchdate = datetime.strftime(datetime.today() - timedelta(days=1), '%m_%d_%Y')
movieids =  pd.read_json(f'http://files.tmdb.org/p/exports/movie_ids_{fetchdate}.json.gz', lines=True)

In [96]:
movieids.loc[~movieids['id'].isin(dbmovieids['id'])]

,adult,id,original_title,popularity,video
764799,False,1063532,Blur,1.4,False
764800,False,1063533,WHY I HATE TIME TRAVEL,1.4,False
764801,False,1063534,楼,1.4,False
764802,False,1063535,Christmas Around the World,0.6,False
764803,False,1063536,Walk-Through,0.6,False
...,...,...,...,...,...
765073,False,1063862,Voldemort: Origins of the Heir,0.0,False
765074,False,1063863,七月返歸,0.0,False
765075,False,1063864,Taevatrepp,0.0,False
765076,False,1063865,Just Above My Head,0.0,False


In [92]:
helper.get_row_count(dbmovieids, movieids)

[764821, 765078]

In [102]:
import json
import requests
from datetime import datetime, timedelta
fetchdate = datetime.strftime(datetime.today() - timedelta(days=1), '%m_%d_%Y')
def getGenres():
    dataurl = f'https://api.themoviedb.org/3/genre/movie/list?api_key={apikey}'
    data = json.loads(requests.get(dataurl).text)
    genres = pd.DataFrame.from_dict(data['genres'])
    return genres

def extractAlltoDB():
    print("Getting Genres")
    with pyodbc.connect(cstring) as conn:
        query = "select id from Genres"
        dbgenres = pd.read_sql(query, conn)
        genres = getGenres()
        genres = genres.query("id not in @dbgenres['id']")   
        
    if genres.shape[0] > 0:
        print(f"Dumping {genres.shape[0]} records into Genres table")
        dataloader.inc_load_with_index(df=genres, tbl="Genres",)
    else:
        print("No addition genres found")

In [103]:
extractAlltoDB()

Getting Genres
Dumping 0 records into Genres table
No addition genres found


/var/folders/74/ndv7v28d3078l2cq22t43pkc0000gn/T/ipykernel_29871/2702218467.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dbgenres = pd.read_sql(query, conn)


In [4]:
imdbratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep='\t')

In [5]:
imdbratings.rename(columns={'tconst':'imdbid', 'averageRating':'rating', 'numVotes':'votes'}, inplace =True)

In [6]:
imdbratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258985 entries, 0 to 1258984
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   imdbid  1258985 non-null  object 
 1   rating  1258985 non-null  float64
 2   votes   1258985 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.8+ MB


In [7]:
imdbratings[imdbratings['imdbid']== 'tt0147800']

,imdbid,rating,votes
103420,tt0147800,7.3,345783


In [9]:
custom = {"imdbid": "varchar(20) PRIMARY KEY",}
dataloader.full_load_with_index(df=imdbratings, tbl="IMDBratings", custom=custom)

DataFrame full loaded to Table: IMDBratings


In [3]:
imdbtitles = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t')

/var/folders/74/ndv7v28d3078l2cq22t43pkc0000gn/T/ipykernel_10882/2475458758.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdbtitles = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t')


In [49]:
imdbtitles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9476134 entries, 0 to 9476133
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 650.7+ MB


In [4]:
imdbtitles = (
        imdbtitles.query("isAdult in [0,1, '0', '1'] and titleType == 'movie'")
                .drop(['startYear', 'endYear'], axis =1)
                .dropna(subset=['primaryTitle', 'originalTitle'], axis=0).replace({'\\N': 0})
                .assign(isAdult = lambda x: x['isAdult'].astype('bool'),
                        runtimeMinutes = lambda x: x['runtimeMinutes'].astype('int'),
                        )
                .rename(columns = {'tconst': 'imdbid'})
                .dropna(how='all')
        )

In [34]:
imdbtitles.describe()

,runtimeMinutes
count,9.476113e+06
mean,1.248839e+01
std,4.943207e+01
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,9.000000e+00
max,5.946000e+04


In [5]:
imdbtitles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 630617 entries, 8 to 9476084
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   imdbid          630617 non-null  object
 1   titleType       630617 non-null  object
 2   primaryTitle    630617 non-null  object
 3   originalTitle   630617 non-null  object
 4   isAdult         630617 non-null  bool  
 5   runtimeMinutes  630617 non-null  int64 
 6   genres          630617 non-null  object
dtypes: bool(1), int64(1), object(5)
memory usage: 34.3+ MB


In [36]:
imdbtitles.columns

Index(['imdbid', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'runtimeMinutes', 'genres'],
      dtype='object')

In [54]:
imdbtitles['titleType'].unique()

array(['movie'], dtype=object)

In [55]:
imdbtitles['titleType'].value_counts()

movie    630617
Name: titleType, dtype: int64

In [56]:
imdbtitles.head()

,imdbid,titleType,primaryTitle,originalTitle,isAdult,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,False,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,False,100,"Documentary,News,Sport"
498,tt0000502,movie,Bohemios,Bohemios,False,100,0
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,False,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,False,90,Drama


In [5]:
custom = {"imdbid": "varchar(20) PRIMARY KEY",}
dataloader.full_load_with_index(df=imdbtitles, tbl="IMDBtitles", custom=custom)

DataFrame full loaded to Table: IMDBtitles
